In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pymc3 as pm 
import pandas as pd

data = pd.read_csv('radon.csv')

county_names = data.county.unique()
county_idx = data['county_code'].values

In [3]:
data[['county', 'log_radon', 'floor']].head()

,county,log_radon,floor
0,AITKIN,0.832909,1.0
1,AITKIN,0.832909,0.0
2,AITKIN,1.098612,0.0
3,AITKIN,0.095310,0.0
4,ANOKA,1.163151,0.0


As we can see, we have multiple radon measurements (log-converted to be on the real line) in a county and whether the measurement has been taken in the basement (floor == 0) or on the first floor (floor == 1). Here we want to test the prediction that radon concentrations are higher in the basement

 In math-speak that model would be:

radoni,c=α+β∗floori,c+ϵ

In [7]:
data.county.unique()

array(['AITKIN', 'ANOKA', 'BECKER', 'BELTRAMI', 'BENTON', 'BIG STONE',
       'BLUE EARTH', 'BROWN', 'CARLTON', 'CARVER', 'CASS', 'CHIPPEWA',
       'CHISAGO', 'CLAY', 'CLEARWATER', 'COOK', 'COTTONWOOD', 'CROW WING',
       'DAKOTA', 'DODGE', 'DOUGLAS', 'FARIBAULT', 'FILLMORE', 'FREEBORN',
       'GOODHUE', 'HENNEPIN', 'HOUSTON', 'HUBBARD', 'ISANTI', 'ITASCA',
       'JACKSON', 'KANABEC', 'KANDIYOHI', 'KITTSON', 'KOOCHICHING',
       'LAC QUI PARLE', 'LAKE', 'LAKE OF THE WOODS', 'LE SUEUR',
       'LINCOLN', 'LYON', 'MAHNOMEN', 'MARSHALL', 'MARTIN', 'MCLEOD',
       'MEEKER', 'MILLE LACS', 'MORRISON', 'MOWER', 'MURRAY', 'NICOLLET',
       'NOBLES', 'NORMAN', 'OLMSTED', 'OTTER TAIL', 'PENNINGTON', 'PINE',
       'PIPESTONE', 'POLK', 'POPE', 'RAMSEY', 'REDWOOD', 'RENVILLE',
       'RICE', 'ROCK', 'ROSEAU', 'SCOTT', 'SHERBURNE', 'SIBLEY',
       'ST LOUIS', 'STEARNS', 'STEELE', 'STEVENS', 'SWIFT', 'TODD',
       'TRAVERSE', 'WABASHA', 'WADENA', 'WASECA', 'WASHINGTON',
       'WATONWAN', '

Fortunately there is a middle ground to both of these extreme views. Specifically, we may assume that while αs and βs are different for each county, the coefficients all come from a common group distribution:

αc∼N(μα,σ2α)
βc∼N(μβ,σ2β)

In [11]:
# takes about 45 minutes
indiv_traces = {}
for county_name in county_names:
    # Select subset of data belonging to county
    c_data = data.loc[data.county == county_name]
    c_data = c_data.reset_index(drop=True)
    
    c_log_radon = c_data.log_radon
    c_floor_measure = c_data.floor.values
    
    with pm.Model() as individual_model:
        # Intercept prior
        a = pm.Normal('alpha', mu=0, sigma=1)
        # Slope prior
        b = pm.Normal('beta', mu=0, sigma=1)
    
        # Model error prior
        eps = pm.HalfCauchy('eps', beta=1)
    
        # Linear model
        radon_est = a + b * c_floor_measure
    
        # Data likelihood
        y_like = pm.Normal('y_like', mu=radon_est, sigma=eps, observed=c_log_radon)

        # Inference button (TM)!
        trace = pm.sample()
        
    indiv_traces[county_name] = trace

In [12]:
indiv_traces

{'AITKIN': <MultiTrace: 2 chains, 1000 iterations, 4 variables>,
 'ANOKA': <MultiTrace: 2 chains, 1000 iterations, 4 variables>,
 'BECKER': <MultiTrace: 2 chains, 1000 iterations, 4 variables>,
 'BELTRAMI': <MultiTrace: 2 chains, 1000 iterations, 4 variables>,
 'BENTON': <MultiTrace: 2 chains, 1000 iterations, 4 variables>,
 'BIG STONE': <MultiTrace: 2 chains, 1000 iterations, 4 variables>,
 'BLUE EARTH': <MultiTrace: 2 chains, 1000 iterations, 4 variables>,
 'BROWN': <MultiTrace: 2 chains, 1000 iterations, 4 variables>,
 'CARLTON': <MultiTrace: 2 chains, 1000 iterations, 4 variables>,
 'CARVER': <MultiTrace: 2 chains, 1000 iterations, 4 variables>,
 'CASS': <MultiTrace: 2 chains, 1000 iterations, 4 variables>,
 'CHIPPEWA': <MultiTrace: 2 chains, 1000 iterations, 4 variables>,
 'CHISAGO': <MultiTrace: 2 chains, 1000 iterations, 4 variables>,
 'CLAY': <MultiTrace: 2 chains, 1000 iterations, 4 variables>,
 'CLEARWATER': <MultiTrace: 2 chains, 1000 iterations, 4 variables>,
 'COOK': <Mult